In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt
import scorecardpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandasql import sqldf

# Data

<span style="color:red">Start of dummy part </span>
    
<span style="color:red">___________________________</span>

In [ ]:
# load germancredit data
smp_full = sc.germancredit()
smp_full['target'] = smp_full['creditability'].apply(lambda x: 1 if x == 'bad' else 0)
smp_full = smp_full.drop(columns = ['creditability'])

# increasing the sample size by duplication
for i in range(5):
    smp_full = pd.concat([smp_full, smp_full])

# preparing df with dates in YYYYMMDD int format
date_index = pd.date_range(start='7/31/2017', end='6/30/2023', freq='M')
date_frame = date_index.to_frame(index=False, name = 'date_col')
date_frame.index = date_frame.index + 1
date_frame['RepDate_End'] = date_frame['date_col'].dt.strftime('%Y%m%d').astype(int)

# assigning random dates
smp_full['RepDate_End'] = np.random.randint(1, 73, smp_full.shape[0])
smp_full['RepDate_End'] = smp_full['RepDate_End'].map(date_frame['RepDate_End'])

In [ ]:
breaks_list = sc.germancredit_breaks_list()
_, bins = sc.woebin(smp_full, y='target', x=list(breaks_list.keys()), breaks_list=breaks_list)

card = sc.germancredit_scorecard_points()

smp_full_score = sc.woebin_ply(smp_full, bins=bins)
smp_full_score['score'] = sc.scorecard_ply(smp_full, card, print_step=0)

<span style="color:red">___________________________</span>

<span style="color:red">End of dummy part</span>

### Development sample (train)

In [ ]:
smp_dev = smp_full_score[smp_full_score['RepDate_End'].between(20170731, 20200630)]

In [ ]:
smp_dev['quarter'] = ' Dev.Sample'

### Recent sample (without outcome period - last year)

In [ ]:
smp_rec = smp_full_score[smp_full_score['RepDate_End'].between(20220731, 20230630)]
smp_rec['quarter'] = smp_rec['RepDate_End'].transform(lambda x: str(x//10000) +'Q'+ str((x//100 % 100 - 1) // 3 + 1)) 

In [ ]:
# only end-quarters are taken into analysis for BEH scorecards (do not run for APP scorecards)
smp_rec = smp_rec[(smp_rec['RepDate_End']//100%100).isin([3,6,9,12])]

### Validation sample (last 2-3 years)

In [ ]:
smp_val = smp_full_score[smp_full_score['RepDate_End'].between(20190731, 20220630)]
smp_val['quarter'] = smp_val['RepDate_End'].transform(lambda x: str(x//10000) +'Q'+ str((x//100 % 100 - 1) // 3 + 1)) 

In [ ]:
# only end-quarters are taken into analysis for BEH scorecards (do not run for APP scorecards)
smp_val = smp_val[(smp_val['RepDate_End']//100%100).isin([3,6,9,12])]

# 1. Testing Scorecard

In [ ]:
# good/bad label
target = 'target'

# date column (e.g. snapshot date or application date)
date = 'quarter'

# raw score
score = 'score'

# list of scorecard variables as named in all datasets (e.g. var_1_scr, ..., var_n_scr)
var_list = []
for i in list(card.keys()):
    if i != 'basepoints':
        var_list.append(i+'_woe')

In [ ]:
# defining samples for gini and psi calculation ([date, score, var_1_scr, ..., var_n_scr] + target for smp_gini) 
smp_gini = pd.concat([smp_dev, smp_val])
smp_psi = pd.concat([smp_val, smp_rec])

In [ ]:
# Bad Rate over time
gini_ot = smp_gini[[date, target]].groupby([date]).agg(['count', 'sum' ])
gini_ot = gini_ot.rename(columns={"count": "Total", "sum": "Bads"})
gini_ot.columns = gini_ot.columns.droplevel(0)
gini_ot['Bad Rate'] = (gini_ot['Bads'] / gini_ot['Total'])
# adding Gini over time
gini_ot['Gini'] = sc.gini_over_time(smp_gini, target, [score], date)

# Gini for vars over time
gini_vars_ot = sc.gini_over_time(smp_gini, target, var_list, date)

# defining score ranges on train sample
_, brk = pd.cut(smp_dev[score], bins=10, retbins=True, duplicates='drop')
brk = brk.round(decimals=2)
brk = list(filter(lambda x: x>np.nanmin(smp_dev[score]) and x<np.nanmax(smp_dev['score']), brk))
brk = [np.nanmin(smp_psi[score])] + sorted(brk) + [np.nanmax(smp_psi[score])]
# applying score ranges on smp_dev and spm_psi
smp_dev['score_range'] = pd.cut(smp_dev[score], bins=brk, include_lowest=False)
smp_psi['score_range'] = pd.cut(smp_psi[score], bins=brk, include_lowest=False)
# PSI over time (score_range) 
psi_ot = sc.psi_over_time(smp_dev, smp_psi, ['score_range'], date)
# PSI for WoE variables over time
psi_vars_ot = sc.psi_over_time(smp_dev, smp_psi, var_list, date)

# calculating hhi over time
smp_hhi = pd.concat([smp_dev, smp_psi])
hhi_ot = smp_hhi.groupby(date).agg({'score_range': sc.hhi})
hhi_ot = hhi_ot.rename(columns={"score_range": "HHI"})

# exporting results to excel
writer = pd.ExcelWriter('1_scorecard_performance.xlsx', engine='xlsxwriter')
gini_ot.to_excel(writer, sheet_name='Gini_OT')
gini_vars_ot.to_excel(writer, sheet_name='Gini_Vars_OT')
psi_ot.to_excel(writer, sheet_name='PSI_OT')
psi_vars_ot.to_excel(writer, sheet_name='PSI_Vars_OT')
hhi_ot.to_excel(writer, sheet_name='HHI_OT')
writer.close()

# 2. Scorecard Recalibration

In [ ]:
# Calibration sample should contain all available history without any exclusion of the periods
smp_calib = smp_full_score

# good/bad label
target = 'target'

# raw score
score = 'score'

In [ ]:
intercept, slope = sc.calibration(smp_calib, score, target)
calib_param = pd.DataFrame({'intercept':pd.Series(intercept),'slope':pd.Series(slope)})
calib_param.reset_index(drop=True).to_excel('2_recalibration.xlsx')
calib_param

# 3. PD estimation

<span style="color:red">Start of dummy part </span>
    
<span style="color:red">___________________________</span>

In [ ]:
smp_calib = smp_full_score
smp_calib['quarter'] = smp_calib['RepDate_End'].transform(lambda x: str(x//10000) +'Q'+ str((x//100 % 100 - 1) // 3 + 1)) 

In [ ]:
# assigning ratings
bins = [0,500,540,580,620,660,700,740,780,1000]
labels = ['4.5','4.0','3.5','3.0','2.5','2.0','1.5','1.0','0.5']
smp_calib['rating'] = pd.cut(smp_calib['score'], bins=bins, labels=labels, include_lowest=True)

# calculating some PDs per rating
smp_pd_prev = smp_calib[smp_calib['RepDate_End'].between(20170731, 20210630)]
pd_est_prev = smp_pd_prev[['rating', 'target']].groupby(['rating']).agg(['count', 'sum' ])
pd_est_prev = pd_est_prev.rename(columns={"count": "Total", "sum": "Bads"})
pd_est_prev.columns = pd_est_prev.columns.droplevel(0)
pd_est_prev['PD'] = (pd_est_prev['Bads'] / pd_est_prev['Total'])

# creating a sample for PD estimation/testing
smp_pd = smp_calib[['quarter', 'rating', 'target']].groupby(['quarter', 'rating']).agg(['count', 'sum' ])
smp_pd = smp_pd.rename(columns={"count": "Total", "sum": "Bads"})
smp_pd.columns = smp_pd.columns.droplevel(0)
smp_pd['DR'] = (smp_pd['Bads'] / smp_pd['Total']).fillna(0)
smp_pd = smp_pd.reset_index()
smp_pd['PD'] = smp_pd['rating'].map(pd_est_prev['PD'])

In [ ]:
smp_pd

<span style="color:red">___________________________</span>

<span style="color:red">End of dummy part</span>

In [ ]:
# smp_pd should contain the following columns - quarter, rating, Total, Bads, DR, PD


In [ ]:
# converting rating to float format
smp_pd['rating_numb'] = smp_pd['rating'].astype(float)

# calculating DR and Gini by rating over time
query = """ select quarter, 
                sum(Total) as Total, 
                sum(Bads) as Bads,
                sum(Bads)*1.00/sum(Total) as DR
            from smp_pd 
            group by quarter
            order by quarter
        """
gini_pd_ot = sqldf(query).set_index('quarter')
gini_pd_ot['Gini'] = -sc.gini_over_time(smp_pd, 'DR', ['rating_numb'], 'quarter', weight='Total')

# calculating PSI by rating comparing to previous period
psi_pd_ot = sc.psi_prev_period_ot(smp_pd, ['rating'], 'quarter', 'Total')

# calculating HHI by rating over time
def hhi_grp(cnt): return np.square(cnt / cnt.sum()).sum()
hhi_pd_ot = smp_pd.groupby('quarter').agg({'Total': hhi_grp})

# new PD estimates
query = """ select rating,
                round(sum(Bads)*1.00/sum(Total),4) as PD
            from smp_pd 
            group by rating
            order by rating
        """
new_PD = sqldf(query).set_index('rating')

# exporting results to excel
writer = pd.ExcelWriter('3_PD_estimation.xlsx', engine='xlsxwriter')
gini_pd_ot.to_excel(writer, sheet_name='Gini_OT')
psi_pd_ot.to_excel(writer, sheet_name='PSI_OT')
hhi_pd_ot.to_excel(writer, sheet_name='HHI_OT')
new_PD.to_excel(writer, sheet_name='New_PD_Estimates')
writer.close()